# lambda と Seifert genus の関係性に関する一考察

## Libraries

In [ ]:
# import random
import re
import ast
import itertools
from tqdm.notebook import tqdm

import math
import numpy as np
import pandas as pd
import seaborn as sb

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

#import sys; sys.path.append("../modules")
#import sympy; sympy.init_printing()
#import Permutation as pm
#from Grid import *

# ----- Debugger -----
# from IPython.core.debugger import Pdb; Pdb().set_trace()
# from knot_floer_homology import *

In [ ]:
def get_heat_matrix(cr_rng, sample=0):
    fname = f"./data/hfk_{cr_rng}.csv"
    try:
        DF = pd.read_csv(fname).set_index('Name'); DF['lambda'] = DF['lambda'].astype('Int64')
        df = DF.query('fibered == True').loc[:, ['lambda', 'seifert_genus']]
        if sample > 0:
            df = df.sample(n=sample, replace = True if len(df) < sample else False)
        #----
        mat = np.zeros([df['lambda'].max()+1,df['seifert_genus'].max()+1], dtype = int)
        for name, values in df.iterrows():
            i, j = values['lambda'], values['seifert_genus']
            mat[i][j] += 1
        #----
    except:
        mat = [[0]]
    return mat

In [ ]:
cr_rngs = [range(3,10),range(10,13),range(13,14),range(14,15),range(15,16),range(16,17)]
Row, Col = 2, 3
fig = make_subplots(Row, Col, subplot_titles=[f"crossing={[*rng]}" for rng in cr_rngs], horizontal_spacing=0.15)
#---
for i, j in itertools.product(range(Row), range(Col)):
    r = i * Col + j
    try:
        mat = get_heat_matrix(cr_rngs[r])
        fig.add_trace(
            go.Heatmap(z = mat,
                       colorscale = 'Oranges',
                       colorbar_len=0.4, colorbar_x=0.25 + 0.385*j, colorbar_y = 0.18 + 0.63*(1-i),                        
            ), 
            i+1, j+1)
    except:
        pass
# ---
fig.update_layout(
    xaxis_title="Seifert genus", yaxis_title="lambda",
    height = 400 * Row, width = 400 * Col, title="heatmaps"
)
# ---
fig.show()

In [ ]:
cr_rngs = [range(3,10),range(10,13),range(13,14),range(14,15),range(15,16),range(16,17)]
Row, Col = 2, 3
fig = make_subplots(Row, Col, subplot_titles=[f"crossing={[*rng]}" for rng in cr_rngs], horizontal_spacing=0.15)
#---
for i, j in itertools.product(range(Row), range(Col)):
    r = i * Col + j
    try:
        mat = get_heat_matrix(cr_rngs[r], sample=1000)
        fig.add_trace(
            go.Heatmap(z = mat,
                       colorscale = 'Oranges',
                       colorbar_len=0.4, colorbar_x=0.25 + 0.385*j, colorbar_y = 0.18 + 0.63*(1-i),                        
            ), 
            i+1, j+1)
    except:
        pass
# ---
fig.update_layout(
    xaxis_title="Seifert genus", yaxis_title="lambda",
    height = 400 * Row, width = 400 * Col, title="heatmaps"
)
# ---
fig.show()

## DataFrame of all knots with 3 to 15 crossings

最小項点数3から15の全ての fibered knot の DataFrame 作成。各行が各 fibered knot. 項目は 'seifert_genus' と 'lambda'.

In [ ]:
cr_rngs = [range(3,10),range(10,13),range(13,14),range(14,15),range(15,16)] #,range(16,17)]

dfs = []
for rng in cr_rngs:
    fname = f"./data/hfk_{rng}.csv"
    DF = pd.read_csv(fname).set_index('Name'); DF['lambda'] = DF['lambda'].astype('Int64')
    dfs.append(DF.query('fibered == True').loc[:, ['lambda', 'seifert_genus']])
df_all = pd.concat(dfs)
display(df_all)

## Frequency distribution

上で作った DataFrame から、大きさ5000で標本抽出して、以下の行列 `mat` を作成。

    `mat`: (i,j)-成分は、('lambda', 'seifert_genus')=(i,j) である fibered knot の個数

この行列を heatmap 表示もしている。

In [ ]:
df_sample = df_all.sample(n=5000)
#----
mat = np.zeros([df_sample['lambda'].max()+1,df_sample['seifert_genus'].max()+1], dtype = int)
for name, values in df_sample.iterrows():
    i, j = values['lambda'], values['seifert_genus']
    mat[i][j] += 1
#----

display(mat)
sb.heatmap(mat, cmap='Oranges')

## Mean of lambda with the same Seifert genus

上の結果を見ると、各 seifert genus において、lamnda の平均値が、seifert genus の値に一致しそうに見える。

そこで、全 fibered knot のデータにおいて、各 seifert genus における lamnda の平均値を表示してみた。

In [ ]:
df_mean_of_lambda = pd.DataFrame(
    { 'seifert_genus': [*range(1,10)], 
     'mean of lambda': [df_all.query(f"seifert_genus == {g}").mean()['lambda'] for g in range(1,10)]
    }).set_index('seifert_genus')

display(df_mean_of_lambda)